In [1]:
import warnings
warnings.filterwarnings("ignore")
import os

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import env, acquire

In [4]:
def get_connection(db, user=env.user, host=env.host, password=env.password):
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'

In [21]:
query = """select calculatedfinishedsquarefeet, bathroomcnt, bedroomcnt, taxvaluedollarcnt
           from properties_2017
           join predictions_2017 using(parcelid)
           where transactiondate between '2017-05-01' and '2017-06-30'
           and propertylandusetypeid in (261, 262, 263, 264, 266, 268, 273, 275, 276, 279)
        """

def get_zillow_data():
    filename = 'zillow.csv'
    
    if os.path.isfile(filename):
        return pd.read_csv(filename, index_col=0)
    else:
        df = pd.read_sql(query, get_connection('zillow'))
        df.to_csv(filename)
        return df

## Data Acquisition

In [2]:
df = acquire.acquire_cache_data()
df.head()

,calculatedfinishedsquarefeet,bathroomcnt,bedroomcnt,taxvaluedollarcnt
0,1458.0,2.0,3.0,136104.0
1,1421.0,1.0,2.0,35606.0
2,2541.0,3.0,4.0,880456.0
3,1650.0,2.0,3.0,614000.0
4,693.0,1.0,2.0,274237.0


## Data Preparation
### Summarize the data
- df.info()
- df.describe()
- df.dtypes

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20364 entries, 0 to 20363
Data columns (total 4 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   calculatedfinishedsquarefeet  20318 non-null  float64
 1   bathroomcnt                   20364 non-null  float64
 2   bedroomcnt                    20364 non-null  float64
 3   taxvaluedollarcnt             20364 non-null  float64
dtypes: float64(4)
memory usage: 636.5 KB


In [6]:
df. describe()

,calculatedfinishedsquarefeet,bathroomcnt,bedroomcnt,taxvaluedollarcnt
count,20318.000000,20364.000000,20364.000000,2.036400e+04
mean,1776.493208,2.284694,3.019839,5.054039e+05
std,936.606498,0.958072,1.030974,6.584038e+05
min,242.000000,0.000000,0.000000,1.050400e+04
25%,1176.000000,2.000000,2.000000,2.104808e+05
50%,1536.000000,2.000000,3.000000,3.665045e+05
75%,2102.000000,3.000000,4.000000,5.890250e+05
max,15450.000000,11.000000,12.000000,2.385837e+07


In [7]:
df.dtypes

calculatedfinishedsquarefeet    float64
bathroomcnt                     float64
bedroomcnt                      float64
taxvaluedollarcnt               float64
dtype: object

### Address missing values

In [14]:
df[df.bathroomcnt == 0].shape

(70, 4)

In [11]:
df[df.bedroomcnt == 0].shape

(136, 4)

In [10]:
df.calculatedfinishedsquarefeet.isnull().sum()

46

**Takeways**
- 70 observations has bathroom count 0
- 136 observations has bedroom count 0
- 46 observations has square feet as null
- How to deal with them?

### Address dupliates

In [23]:
df.duplicated().sum()

61

In [24]:
df.drop_duplicates(keep='first', ignore_index=True)

,calculatedfinishedsquarefeet,bathroomcnt,bedroomcnt,taxvaluedollarcnt
0,1458.0,2.0,3.0,136104.0
1,1421.0,1.0,2.0,35606.0
2,2541.0,3.0,4.0,880456.0
3,1650.0,2.0,3.0,614000.0
4,693.0,1.0,2.0,274237.0
...,...,...,...,...
20298,1030.0,2.0,2.0,359829.0
20299,1536.0,2.0,3.0,297097.0
20300,2655.0,3.0,5.0,746963.0
20301,2305.0,3.0,4.0,579047.0


**Takeways**
- 61 obervations were duplicates
- How to deal with them?